
 w# Cloud-Native Monitoring with EBPF

gergely.szabo@origoss.com

# About Myself
* more than 15 years in the industry
* research, development, system architect, etc...
* currently at Origoss Solutions
  * Cloud Native
  * Kubernetes
  * Prometheus

# Agenda
* BPF
* EBPF
* Linux Kernel Tracing
* Prometheus Exporter

# BPF

# Packet Filtering Problem
  
![network filtering](filter.svg "Network filtering")

# Filtering Requirements

* Efficient
* Flexible filter rules
* Safe

# Where to Filter?

![Where to filter?](filter2.svg "Where to Filter?")

# BPF

## Steven McCanne and Van Jacobson:

## The BSD Packet Filter: A New Architecture for User-level Packet Capture, 1992

http://www.tcpdump.org/papers/bpf-usenix93.pdf